# Getting Started With the SHFPPC

## Device Setup and Data Server

The SHFPPC instrument is intended to be used together with an SHFQA+ or SHFQC+ instrument. Similar to a travelling-wave parametric amplifier (TWPA), the SHFPPC is an optional element added to the signal path connected to the acquire line of a QPU.
While the TWPA is added to the acquire-signal path at cryogenic temperatures, providing amplification of the measurement signal that returns from the QPU, the SHFPPC is added to the same signal path at room temperature to cancel the TWPA pump tone in the measurement signal before the signal is recorded with the QA. To learn more about how to connect the SHFPPC to an SHFQA+ instrument, have a look at [the tutorials](https://docs.zhinst.com/shfppc_user_manual/tutorials/index.html) in the user's manual for the SHFPPC.

Let's start by creating a `DeviceSetup` with a SHFPPC instrument and an SHFQA+. To learn more about what a `DeviceSetup` is and how to use it, have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/index.html).

First, we import `laboneq.simple`, which contains the `DeviceSetup`.

In [ ]:
from laboneq.simple import *

Create the `DeviceSetup` and add the information about the data server it should connect to. Note that calling `add_dataserver` does not attempt the connection yet. This will be done only upon calling `Session.connect`.

In [ ]:
device_setup = DeviceSetup("ZI_SHFPPC_SHFQA")
device_setup.add_dataserver(
    host="localhost",
    port="8004",
)

Create the `SHFQA` and `SHFPPC` instruments, and add them to the `DeviceSetup`. Assign unique identifiers (`uid`) of your choice to each instrument, and specify the device names under `address`. For the SHFQA, specify the options that are installed on your instrument. The possible options you can set for an SHFQA instrument are:

* either `"SHFQA4"` if you have a 4-channel instrument or `"SHFQA2"` for a 2-channel version;
* `PLUS` if you have the SHFQA+;
* `"16W"` available only in combination with `"SHFQA2"`to extend the avilable integration weights to 16 per channel.

When passing these options to the instrument, add them in any order, separated by a forward slash (`"/"`). Below, we will use `device_options="SHFQA4/PLUS"`. Have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/02_instrument_options.html?h=instrument+options) to learn more about the possible options of our instruments.


You can also pass additional input parameters to configure your instruments:

* the `interface` over which to connect to the instrument, either `"1GbE"` (default) or `"usb"`. Note that **to ensure the stability of the connection to the instrument, we recommend to use the ethernet interface instead of USB**;

* (only for the SHFQA) the `reference_clock_source` for the instrument, either as `"internal"` to use the instrument's own internal reference clock, or `"external"` (default) if you are using an external source like the PQSC instrument, for example.

In [ ]:
device_setup.add_instruments(
    SHFQA(
        uid="shfqa",
        address="dev12300",
        interface="1GbE",
        device_options="SHFQA4/PLUS/16W",
        reference_clock_source="internal",
    )
)
device_setup.add_instruments(
    SHFPPC(
        uid="shfpcc",
        interface="1GbE",
        address="dev12301",
    )
)

Next, we create connections from each of the four I/O ports of the SHFPPC to the four input ports of the SHFQA+ instrument, and add these connections to the `DeviceSetup`. These connections are represented in LabOne Q as [logical signal lines](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/02_logical_signals/concepts/index.html) between the instruments and your device under test. Here, we assume the device under test is a six-qubit QPU and use the physical lines of these qubits as a naming convention for our signal lines. Note that the instances of the `LogicalSignal`s will be created automatically by the `add_connections` method of `DeviceSetup`, with the names that we have specified.

In [ ]:
for idx in range(4):
    device_setup.add_connections(
        "shfqa",
        create_connection(
            to_signal=f"twpa{idx}/measure",
            ports=f"QACHANNELS/{idx}/OUTPUT",
        ),
        create_connection(
            to_signal=f"twpa{idx}/acquire", ports=f"QACHANNELS/{idx}/INPUT"
        ),
    )
    device_setup.add_connections(
        "shfpcc",
        create_connection(to_signal=f"twpa{idx}/acquire", ports=f"PPCHANNELS/{idx}"),
    )

You can inspect the `LogicalSignal`s that have been created by calling `device_setup.logical_signal_by_uid(signal_name)`; for example:

In [ ]:
device_setup.logical_signal_by_uid("twpa0/measure")

Next, we need to configure the `calibration` of the signal lines of the `DeviceSetup`. Next, we need to configure the `calibration` of the signal lines of the `DeviceSetup`. We will set the most common properties for the measure and acquire lines:

* `range` - the power range in dBm of the input and output ports;
* `local_oscillator` - an instance of `Oscillator` where we specify the local oscillator frequency;
* `oscillator` - an instance of `Oscillator` where we specify the IF frequency that will be mixed with local oscillator frequency.

Keep in mind that the channels 0-1 and 2-3 share the local oscillator, so the local oscillator for these channels must be configured with the same frequency value.

Configuring the SHFPPC usually means setting the pump frequency and power, the probe-tone frequency, and the cancellation properties. Below, we set the same values to these parameters for all 4 acquire lines.

In [ ]:
qa_lo_0 = Oscillator(frequency=7e9)
qa_lo_1 = Oscillator(frequency=7.2e9)
pump_frequency = 8e9

config = Calibration()
for idx, qa_lo in enumerate([qa_lo_0, qa_lo_1]):
    config[f"q{2 * idx}/measure"] = SignalCalibration(
        local_oscillator=qa_lo, oscillator=Oscillator(frequency=100e6), range=5
    )
    config[f"q{2 * idx + 1}/measure"] = SignalCalibration(
        local_oscillator=qa_lo, oscillator=Oscillator(frequency=100e6), range=5
    )

    config[f"q{2 * idx}/acquire"] = SignalCalibration(
        local_oscillator=qa_lo,
        oscillator=Oscillator(frequency=100e6),
        range=10,
        amplifier_pump=AmplifierPump(
            pump_frequency=pump_frequency,
            pump_power=12.5,
            pump_on=True,
            pump_filter_on=True,
            cancellation_on=False,
            cancellation_phase=0,
            cancellation_attenuation=10,
            cancellation_source=CancellationSource.INTERNAL,
            cancellation_source_frequency=pump_frequency,
            alc_on=True,
            probe_on=False,
            probe_frequency=6.8e9,
            probe_power=0,
        ),
    )
    config[f"q{2 * idx + 1}/acquire"] = SignalCalibration(
        local_oscillator=qa_lo,
        oscillator=Oscillator(frequency=100e6),
        range=10,
        amplifier_pump=AmplifierPump(
            pump_frequency=pump_frequency,
            pump_power=12.5,
            pump_on=True,
            pump_filter_on=True,
            cancellation_on=False,
            cancellation_phase=0,
            cancellation_attenuation=10,
            cancellation_source=CancellationSource.INTERNAL,
            cancellation_source_frequency=pump_frequency,
            alc_on=True,
            probe_on=False,
            probe_frequency=6.8e9,
            probe_power=0,
        ),
    )

# Apply the configuration to the DeviceSetup
device_setup.set_calibration(config)

Great! We have our `DeviceSetup` for a system containing an SHFPPC and an SHFQA+.

Before we can play a signal on the instrument, we first have to connect it to the LabOne data server via the LabOne Q [Session](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/index.html). Here, we connect in emulation mode by calling `Session.connect` with `do_emulation=True`. Set this flag to False in order to connect to a physical setup.

In [ ]:
session = Session(device_setup)
session.connect(do_emulation=True)  # do_emulation=False when at a physical setup

## Scan Pump Parameters Experiment

Let's create a simple experiment that sweeps the power and frequency on all 4 of the the pump output ports of the SHFPPC. We also configure the 4 corresponding SHFQA+ input channels to perform an integrated average acquisition using a constant 2-$\mu$s integration kernel.

In [ ]:
import numpy as np


@dsl.experiment(
    signals=[f"twpa{idx}_measure" for idx in range(4)]
    + [f"twpa{idx}_acquire" for idx in range(4)]
)
def scan_pump_parameters_experiment(count):
    with dsl.acquire_loop_rt(
        count=count,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        with dsl.sweep(
            name="power_sweep",
            parameter=SweepParameter("pump_powers", np.linspace(8, 16, 3)),
        ) as power:
            with dsl.sweep(
                name="frequency_sweep",
                parameter=SweepParameter("pump_frequencies", np.linspace(6e9, 7e9, 11)),
            ) as frequency:
                for idx in range(4):
                    with dsl.section(name=f"acquire-twpa{idx}"):
                        dsl.acquire(
                            f"twpa{idx}_acquire",
                            handle=f"twpa{idx}/result",
                            kernel=pulse_library.const(amplitude=1, length=2e-6),
                        )
                        # add a processing delay
                        dsl.delay(f"twpa{idx}_acquire", time=0.5e-6)

    # Set the pump frequency and power sweeps in the experiment calibration
    exp_calibration = dsl.experiment_calibration()
    for idx in range(4):
        exp_calibration[f"twpa{idx}_acquire"] = SignalCalibration(
            amplifier_pump=AmplifierPump(pump_frequency=frequency, pump_power=power)
        )

        # Map the ExperimentSignals "q{idx}_measure", "q{idx}_acquire" to the logical signal lines defined in the `DeviceSetup`
        dsl.map_signal(f"twpa{idx}_measure", f"twpa{idx}/measure")
        dsl.map_signal(f"twpa{idx}_acquire", f"twpa{idx}/acquire")

Next, we instantiate the `Experiment` by running the function `simple_experiment`.

In [ ]:
experiment = scan_pump_parameters_experiment(5)

Note that, the mapping between the `ExperimentSignal`s and the logical signal lines do not have to be done as part of `simple_experiment` but can also be done or modified on the `Experiment` instance returned by `simple_experiment` as follows:

```python
twpa0_ls = device_setup.logical_signal_groups["twpa0"].logical_signals
experiment.map_signal("measure", twpa0_ls["measure"])
experiment.map_signal("acquire", twpa0_ls["acquire"])
```

You can also map an `ExperimentSignal` directly to a `LogicalSignal` instead of its UID, as written above.

Compile the simple_experiment and inspect it using the pulse-sheet viewer:

In [ ]:
compiled_experiment = session.compile(experiment)

You can inspect your experiment by using the interactive pulse-sheet viewer. Calling the function `show_pulse_sheet` creates an HTML file that you can open in your browser. To show the pulse sheet in the kernel, set `interactive=True`.

In [ ]:
show_pulse_sheet("scan_pump_parameters", compiled_experiment, interactive=True)

Finally, let's run the experiment on the instrument.

In [ ]:
results = session.run()

You can inspect the acquired results for any of the handle names that we've specified when defining the pulse sequence; for example, `twpa0/result`:

In [ ]:
results["twpa0/result"]

Or, alternatively,

In [ ]:
results.twpa0.result